In [1]:
%matplotlib inline

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Chevron/training.csv')

df.head()

,StoreNumber,dayOfTheYear,3HourBucket,GrossSoldQuantity,Cash/Credit Site,EBT Site,Loyalty Site,ExtraMile Site,CoBrand,Alcohol,Carwash,Food Service,City,State
0,1000,1,1,3,False,True,True,True,No,True,False,True,HOUSTON,TX
1,1000,1,2,5,False,True,True,True,No,True,False,True,HOUSTON,TX
2,1000,1,3,6,False,True,True,True,No,True,False,True,HOUSTON,TX
3,1000,1,4,3,False,True,True,True,No,True,False,True,HOUSTON,TX
4,1000,2,1,13,False,True,True,True,No,True,False,True,HOUSTON,TX


We see that all attributes, excluding dayOfTheYear and GrossSoldQuantity, can be represented as categorical
attributes.

In [3]:
corr = df.corr()
corr['GrossSoldQuantity']

StoreNumber          0.419910
dayOfTheYear        -0.009997
3HourBucket          0.239332
GrossSoldQuantity    1.000000
Cash/Credit Site          NaN
EBT Site            -0.297379
Loyalty Site              NaN
ExtraMile Site            NaN
Alcohol             -0.297379
Carwash              0.320785
Food Service              NaN
Name: GrossSoldQuantity, dtype: float64

Looking at strictly linear correlations, we see there exists a non-negligible correlation between GrossSoldQuantity and StoreNumber, 3HourBucket, EBT Site, Alcohol, and Carwash. The remaining have only one unique value and therefore afford no predictive power.

In [7]:
# before exploring our data further, let's create a stratified train-test split

from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in sss.split(df, df['StoreNumber']):
    # respective indices are then assigned to training and testing sets
    sss_train_set = df.loc[train_index]
    sss_test_set = df.loc[test_index]
    
explore = sss_train_set.copy()

# get rid of outliers
explore = explore[(np.abs(stats.zscore(explore['GrossSoldQuantity'])) < 3)]
# add day of week attribute
explore['dayOfWeek'] = explore['dayOfTheYear'].apply(lambda x: x%7)


y_train = explore['GrossSoldQuantity'].copy()
X_train = explore.drop('GrossSoldQuantity', axis = 1)
y_test = sss_test_set['GrossSoldQuantity'].copy()
X_test = sss_test_set.drop('GrossSoldQuantity', axis = 1)

X_test['dayOfWeek'] = X_test['dayOfTheYear'].apply(lambda x: x%7)


corr = explore.corr()
corr['GrossSoldQuantity']

StoreNumber          0.424648
dayOfTheYear         0.005881
3HourBucket          0.231983
GrossSoldQuantity    1.000000
Cash/Credit Site          NaN
EBT Site            -0.290886
Loyalty Site              NaN
ExtraMile Site            NaN
Alcohol             -0.290886
Carwash              0.327494
Food Service              NaN
dayOfWeek            0.082354
Name: GrossSoldQuantity, dtype: float64

We see that our dayOfWeek attribute affords more predictive power as opposed to dayOfTheYear, so we will keep the former and remove the latter in our data preprocessing pipeline

In [8]:
# let's begin transforming our categorical attributes using OneHotEncoding

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

# pick out only a few promising categorical attributes
cat_attrs = ['3HourBucket', 'Alcohol', 'dayOfWeek', 'StoreNumber', 'EBT Site', 'Carwash']

explore_cat = explore[cat_attrs].copy()

explore_cat_encoded = encoder.fit_transform(explore_cat)

In [49]:
# create pipeline

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures

# we will use Polynomial Feature Expansion to capture interactions between attributes, but will limit to degree 2
# to avoid overfitting.

full_pipeline = ColumnTransformer([('cat', OneHotEncoder(), cat_attrs),
                                   ('poly', PolynomialFeatures(degree = 2), cat_attrs)])

train_prepared = full_pipeline.fit_transform(X_train)
train_prepared.shape

(4485, 49)

Let's first try a linear regression model

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

scores = cross_val_score(lin_reg, train_prepared, y_train, scoring = 'neg_mean_squared_error', cv=10)

np.sqrt(-scores).mean()

10.901477725398646

Now let's try a ridge linear regression model with various alpha values

In [20]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score

for alpha in range(1, 50, 1):

    ridge = Ridge(alpha = alpha)
    
    scores = cross_val_score(ridge, train_prepared, y_train, scoring = 'neg_mean_squared_error', cv=10)

    print('RSME average is {} with alpha value {}'.format(np.sqrt(-scores).mean(), alpha))

RSME average is 10.901448902918329 with alpha value 1
RSME average is 10.901435070662265 with alpha value 2
RSME average is 10.901436088262201 with alpha value 3
RSME average is 10.90145181658922 with alpha value 4
RSME average is 10.90148211782616 with alpha value 5
RSME average is 10.901526855454076 with alpha value 6
RSME average is 10.901585894238918 with alpha value 7
RSME average is 10.901659100218227 with alpha value 8
RSME average is 10.901746340688106 with alpha value 9
RSME average is 10.901847484190293 with alpha value 10
RSME average is 10.901962400499345 with alpha value 11
RSME average is 10.902090960610018 with alpha value 12
RSME average is 10.902233036724782 with alpha value 13
RSME average is 10.902388502241436 with alpha value 14
RSME average is 10.902557231740902 with alpha value 15
RSME average is 10.902739100975134 with alpha value 16
RSME average is 10.9029339868552 with alpha value 17
RSME average is 10.903141767439426 with alpha value 18
RSME average is 10.9033

Let's now try a KNN Regression model with weights set to uniform

In [24]:
# now let's try knn with weights = 'uniform'

from sklearn.neighbors import KNeighborsRegressor

for neighbors in range(1, 50, 2):

    knn = KNeighborsRegressor(n_neighbors = neighbors, weights = 'uniform')

    scores = cross_val_score(knn, train_prepared, y_train, scoring = 'neg_mean_squared_error', cv=10)
    np.sqrt(-scores).mean()
    print('RSME average is {} with neighbor count {}'.format(np.sqrt(-scores).mean(), neighbors))
    
# best value so far is 9.744967139562855 and n_neighbors = 34

RSME average is 13.082309380734099 with neighbor count 1
RSME average is 10.579237895772916 with neighbor count 3
RSME average is 10.346528370233903 with neighbor count 5
RSME average is 10.145164578045184 with neighbor count 7
RSME average is 10.026819953093643 with neighbor count 9
RSME average is 9.99421845883565 with neighbor count 11
RSME average is 9.884782716448058 with neighbor count 13
RSME average is 9.827166002748676 with neighbor count 15
RSME average is 9.80286434747117 with neighbor count 17
RSME average is 9.80522371653308 with neighbor count 19
RSME average is 9.755791348337823 with neighbor count 21
RSME average is 9.751407793404448 with neighbor count 23
RSME average is 9.73614526042476 with neighbor count 25
RSME average is 9.766636707853243 with neighbor count 27
RSME average is 9.755508727140533 with neighbor count 29
RSME average is 9.744446382206885 with neighbor count 31
RSME average is 9.743878964950904 with neighbor count 33
RSME average is 9.745761534865377 w

Now with weights set to distance

In [25]:
# knn with weights = 'distance'

for neighbors in range(1, 50, 3):

    knn = KNeighborsRegressor(n_neighbors = neighbors, weights = 'distance')

    scores = cross_val_score(knn, train_prepared, y_train, scoring = 'neg_mean_squared_error', cv=10)
    np.sqrt(-scores).mean()
    print('RSME average is {} with neighbor count {}'.format(np.sqrt(-scores).mean(), neighbors))

RSME average is 13.082309380734099 with neighbor count 1
RSME average is 10.573528268633117 with neighbor count 4
RSME average is 10.145164578045184 with neighbor count 7
RSME average is 10.005861360289511 with neighbor count 10
RSME average is 9.884782716448058 with neighbor count 13
RSME average is 9.818526079037728 with neighbor count 16
RSME average is 9.80522371653308 with neighbor count 19
RSME average is 9.7543537666796 with neighbor count 22
RSME average is 9.73897866131987 with neighbor count 25
RSME average is 9.76072124846753 with neighbor count 28
RSME average is 9.755066431975523 with neighbor count 31
RSME average is 9.744531131413854 with neighbor count 34
RSME average is 9.736444892307123 with neighbor count 37
RSME average is 9.733580506800838 with neighbor count 40
RSME average is 9.73663803563289 with neighbor count 43
RSME average is 9.73667913964697 with neighbor count 46
RSME average is 9.73667913964697 with neighbor count 49


In [54]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.base import clone

params = {'learning_rate': np.linspace(0.01, 0.5, 10), 'n_estimators':[i*10 for i in range(1, 10)],
         'max_depth': [3, 4, 5, 6, 7], 'max_features': [5, 10, 15, 20]}

model = GradientBoostingRegressor(random_state = 42)

grid_search = GridSearchCV(model, params, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

grid_search.fit(train_prepared, y_train)

cv = grid_search.cv_results_

for scores, params in sorted(zip(cv['mean_test_score'], cv['params']), reverse = True):
                             print(scores, params)
        
gbr_model = clone(grid_search.best_estimator_)

-9.709607929973583 {'learning_rate': 0.06444444444444444, 'max_depth': 5, 'max_features': 10, 'n_estimators': 70}
-9.711500933271967 {'learning_rate': 0.06444444444444444, 'max_depth': 5, 'max_features': 10, 'n_estimators': 60}
-9.71623810598136 {'learning_rate': 0.22777777777777777, 'max_depth': 4, 'max_features': 10, 'n_estimators': 20}
-9.717179186525229 {'learning_rate': 0.06444444444444444, 'max_depth': 5, 'max_features': 10, 'n_estimators': 80}
-9.71781344518076 {'learning_rate': 0.22777777777777777, 'max_depth': 4, 'max_features': 10, 'n_estimators': 30}
-9.718152604598593 {'learning_rate': 0.33666666666666667, 'max_depth': 6, 'max_features': 20, 'n_estimators': 10}
-9.718241278322562 {'learning_rate': 0.2822222222222222, 'max_depth': 4, 'max_features': 5, 'n_estimators': 30}
-9.718351855532903 {'learning_rate': 0.33666666666666667, 'max_depth': 5, 'max_features': 20, 'n_estimators': 10}
-9.719813360644686 {'learning_rate': 0.11888888888888888, 'max_depth': 6, 'max_features': 20

-9.782416200431978 {'learning_rate': 0.11888888888888888, 'max_depth': 6, 'max_features': 10, 'n_estimators': 90}
-9.78242709579647 {'learning_rate': 0.3911111111111111, 'max_depth': 4, 'max_features': 15, 'n_estimators': 80}
-9.782473580245203 {'learning_rate': 0.3911111111111111, 'max_depth': 4, 'max_features': 20, 'n_estimators': 60}
-9.782484797715902 {'learning_rate': 0.17333333333333334, 'max_depth': 6, 'max_features': 5, 'n_estimators': 70}
-9.78250343698217 {'learning_rate': 0.44555555555555554, 'max_depth': 6, 'max_features': 15, 'n_estimators': 20}
-9.782519539847168 {'learning_rate': 0.3911111111111111, 'max_depth': 7, 'max_features': 5, 'n_estimators': 20}
-9.782530153444828 {'learning_rate': 0.33666666666666667, 'max_depth': 5, 'max_features': 5, 'n_estimators': 60}
-9.782594740490307 {'learning_rate': 0.33666666666666667, 'max_depth': 4, 'max_features': 15, 'n_estimators': 90}
-9.782619964444288 {'learning_rate': 0.33666666666666667, 'max_depth': 4, 'max_features': 20, 'n

-10.602509119245344 {'learning_rate': 0.11888888888888888, 'max_depth': 3, 'max_features': 15, 'n_estimators': 20}
-10.614626550492108 {'learning_rate': 0.22777777777777777, 'max_depth': 4, 'max_features': 5, 'n_estimators': 10}
-10.639515514256992 {'learning_rate': 0.17333333333333334, 'max_depth': 3, 'max_features': 5, 'n_estimators': 20}
-10.69105878084299 {'learning_rate': 0.11888888888888888, 'max_depth': 3, 'max_features': 10, 'n_estimators': 20}
-10.703498951146038 {'learning_rate': 0.06444444444444444, 'max_depth': 3, 'max_features': 5, 'n_estimators': 50}
-10.706098343232952 {'learning_rate': 0.22777777777777777, 'max_depth': 3, 'max_features': 15, 'n_estimators': 10}
-10.736652321268839 {'learning_rate': 0.22777777777777777, 'max_depth': 3, 'max_features': 10, 'n_estimators': 10}
-10.77453941401819 {'learning_rate': 0.2822222222222222, 'max_depth': 3, 'max_features': 5, 'n_estimators': 10}
-10.815259546527695 {'learning_rate': 0.06444444444444444, 'max_depth': 7, 'max_feature

Lastly, let's try a kernelized SVM

In [1]:
# from sklearn.svm import SVR

# params = {'C': [i for i in range(1, 15)], 'gamma': np.linspace(0.01, 20, 10)}

# model = SVR()

# grid_search = GridSearchCV(model, params, scoring = 'neg_root_mean_squared_error', n_jobs = -1)

# grid_search.fit(train_prepared, y_train)

# cv = grid_search.cv_results_

We see that GradientBoostingRegressor is our best performing model, so this is the model we will be using for our validation data and testing data

In [61]:
# model performance on validation data

from sklearn.metrics import mean_squared_error

final_model = gbr_model

final_model.fit(train_prepared, y_train)

test_prepared = full_pipeline.transform(X_test)

predictions = final_model.predict(test_prepared)

np.sqrt(mean_squared_error(y_test, predictions))

11.061255858760434

In [62]:
# final performance on testing data

testing = pd.read_csv('Chevron/testing.csv')
y_test = testing['GrossSoldQuantity']
testing.drop('GrossSoldQuantity', axis = 1, inplace = True)
testing['dayOfWeek'] = testing['dayOfTheYear'].apply(lambda x: x%7)

final_model.fit(train_prepared, y_train)

testing_prepared = full_pipeline.transform(testing)


predictions = final_model.predict(testing_prepared)

np.sqrt(mean_squared_error(y_test, predictions))

6.035813754901408